In [ ]:
# THIS FUNCTION TRAIN ON DIFFERENT MODELS, GET THE SCORE OF EACH MODELS AND CREATE A DICTIONARY OF MODELS AND RESPECTIVE SCORES.


def model_train(df_X,df_Y, metric):
    !pip install lightgbm
    !pip install catboost
    !pip install xgboost
    !pip install rgf_python
    Models=dict()
    from sklearn.metrics import metric 
    from sklearn.model_selection import train_test_split
    from catboost import CatBoostClassifier
    from lightgbm import LGBMClassifier
    from xgboost import XGBClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from rgf.sklearn import RGFClassifier
    from sklearn.neural_network import MLPClassifier
    x_train, x_test, y_train, y_test= train_test_split(df_X, df_Y, test_size=0.2, random_state=42)
    catboost=CatBoostClassifier()
    lightgbm=LGBMClassifier()
    xgb=XGBClassifier()
    Lr=LogisticRegression()
    Rf=RandomForestClassifier()
    Gb=GradientBoostingClassifier()
    rgf=RGFClassifier()
    mlp=MLPClassifier()
    Model_list=[catboost, lightgbm, xgb, Lr, Rf, Gb, rgf, mlp]
    for model in Model_list:
        len_of_model=len(Model_list)
        current=(Model_list.index(model)) + 1
        print ("Training model " + str(current) +  " of " + str(len_of_model)  )
        model.fit(x_train, y_train)
        y_pred=model.predict(x_test)
        sc=metric(y_test, y_pred)
        Models[model]=sc
    print (Models)

Best models from model_train function goes into hyperopt function in list format

In [ ]:
!pip install hyperopt
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.pyll.base import scope

def optimization(Models, X, y):
    for model in Models:
        len_of_model=len(Models)
        current=(Models.index(model)) + 1
        print ("Running hyperopt on " + str(current) +  " of " + str(len_of_model)  )
        def hyperopt(params):
            clf = model(**params)
            cross_val_score(clf, X, y, cv=3).mean()

        space = {
         "n_estimators": hp.choice("n_estimators", [100, 200, 300, 400,500,600]),
          "max_depth": hp.quniform("max_depth", 1, 15,1),
            }

        def f(params):
            acc = hyperopt(params)
            return {'loss': -acc, 'status': STATUS_OK}

        trials = Trials()
        best = fmin(f, space, algo=tpe.suggest, max_evals=3, trials=trials)
        print ('best:')
        print (best)